In [1]:
import pandas as pd
from datetime import date, timedelta
from sqlalchemy import create_engine

engine = create_engine('mysql+pymysql://root:@localhost:3306/portfolio_development')
conpf = engine.connect()

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"

year = 2022

In [2]:
format_dict = {'qty':'{:,}',
              'sell_price':'{:,.2f}','buy_price':'{:,.2f}','diff':'{:,.2f}',
              'sell_amt':'{:,.2f}','buy_amt':'{:,.2f}','gross':'{:,.2f}','profit':'{:,.2f}',
              'pct':'{:,.2f}%','net':'{:,.2f}',
              'cost_amt':'{:,.2f}','unit_cost':'{:,.2f}','avg_cost':'{:,.2f}'}

pd.set_option('display.float_format','{:,.2f}'.format)

### Record selection for transactions

In [3]:
name = 'SIS'

In [29]:
sql = '''
SELECT name, YEAR(sells.date) AS sell_year, MONTH(sells.date) AS sell_month,  
             YEAR(buys.date) AS buy_year, MONTH(buys.date) AS buy_month,
sells.price AS sell_price, buys.price AS buy_price, 
(sells.price - buys.price) AS diff, qty, 
(sells.price * qty) AS sell_amt,
(buys.price * qty) AS buy_amt,
(sells.price - buys.price) * qty AS gross, 
ROUND((sells.price - buys.price)/buys.price*100,2) AS pct, profit, buys.kind
FROM sells JOIN buys ON sells.buy_id = buys.id
JOIN stocks ON buys.stock_id = stocks.id
WHERE name = '%s'
ORDER BY sell_month DESC, name'''

sql = sql % (name)
sells_df = pd.read_sql(sql, conpf)
print(sql)


SELECT name, YEAR(sells.date) AS sell_year, MONTH(sells.date) AS sell_month,  
             YEAR(buys.date) AS buy_year, MONTH(buys.date) AS buy_month,
sells.price AS sell_price, buys.price AS buy_price, 
(sells.price - buys.price) AS diff, qty, 
(sells.price * qty) AS sell_amt,
(buys.price * qty) AS buy_amt,
(sells.price - buys.price) * qty AS gross, 
ROUND((sells.price - buys.price)/buys.price*100,2) AS pct, profit, buys.kind
FROM sells JOIN buys ON sells.buy_id = buys.id
JOIN stocks ON buys.stock_id = stocks.id
WHERE name = 'SIS'
ORDER BY sell_month DESC, name


In [30]:
sells_df.dtypes

name           object
sell_year       int64
sell_month      int64
buy_year        int64
buy_month       int64
sell_price    float64
buy_price     float64
diff          float64
qty             int64
sell_amt      float64
buy_amt       float64
gross         float64
pct           float64
profit        float64
kind           object
dtype: object

In [31]:
sells_df.groupby(['sell_year','sell_month','buy_year','buy_month','name']).gross.sum()

sell_year  sell_month  buy_year  buy_month  name
2021       2           2021      1          SIS    24,600.00
           6           2021      5          SIS     8,750.00
           11          2021      5          SIS    12,000.00
                                 9          SIS    27,250.00
           12          2021      9          SIS     9,000.00
                                 11         SIS    20,250.00
2022       1           2021      11         SIS     3,375.00
           3           2022      1          SIS     1,125.00
           5           2022      5          SIS     1,600.00
Name: gross, dtype: float64

In [34]:
sold_grp = sells_df.groupby(['name','sell_year'])
sold_stocks = sold_grp['sell_amt','buy_amt','qty','gross'].sum()
sold_stocks.sort_values(['name','sell_year'],ascending=[True,True]).style.format(format_dict)

C:\Users\User\AppData\Local\Temp/ipykernel_18744/1810971311.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  sold_stocks = sold_grp['sell_amt','buy_amt','qty','gross'].sum()


In [35]:
sold_stocks['sell_price'] = sold_stocks['sell_amt'] / sold_stocks['qty']
sold_stocks['buy_price'] = sold_stocks['buy_amt'] / sold_stocks['qty']
cols = 'sell_amt buy_amt gross qty sell_price buy_price'.split()
sold_stocks[cols].sort_values(['name'],ascending=[True]).style.format(format_dict)

### Record selection for active stocks

In [36]:
sql = '''
SELECT name, YEAR(buys.date) AS year, MONTH(buys.date) AS month,
buys.price AS unit_cost, qty, 
(buys.price * qty) AS cost_amt
FROM buys
JOIN stocks ON buys.stock_id = stocks.id
WHERE status = 'Active'
AND name = '%s'
ORDER BY name, buys.date'''
sql = sql % name
buys_df = pd.read_sql(sql, conpf)
buys_df.style.format(format_dict)

,name,year,month,unit_cost,qty,cost_amt
0,SIS,2022,5,28.75,"2,800","80,500.00"


In [38]:
buys_df.groupby(['year','month']).cost_amt.sum()

year  month
2022  5       80,500.00
Name: cost_amt, dtype: float64

In [39]:
buys_df.cost_amt.sum()

80500.0

In [40]:
buys_df.groupby(['name'])['cost_amt','qty'].sum().style.format(format_dict)

C:\Users\User\AppData\Local\Temp/ipykernel_18744/3973236850.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  buys_df.groupby(['name'])['cost_amt','qty'].sum().style.format(format_dict)


,cost_amt,qty
name,,
SIS,"80,500.00","2,800"


In [41]:
buys_grp = buys_df.groupby(by=['name'])
dtd_stocks = buys_grp['cost_amt','qty'].sum()
dtd_stocks['avg_cost'] = dtd_stocks['cost_amt'] / dtd_stocks['qty']
dtd_stocks.sort_values(['name'],ascending=[True]).style.format(format_dict)

C:\Users\User\AppData\Local\Temp/ipykernel_18744/726847341.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  dtd_stocks = buys_grp['cost_amt','qty'].sum()


,cost_amt,qty,avg_cost
name,,,
SIS,"80,500.00","2,800",28.75


In [42]:
file_name = 'unit-cost-name.csv'
data_file = data_path + file_name
csv_file = csv_path + file_name
box_file = box_path + file_name
data_file, csv_file, box_file

('../data/unit-cost-name.csv',
 '\\Users\\User\\iCloudDrive\\unit-cost-name.csv',
 '\\Users\\User\\Dropbox\\unit-cost-name.csv')

In [43]:
dtd_stocks.sort_values(['name'],ascending=[True]).to_csv(csv_file)
dtd_stocks.sort_values(['name'],ascending=[True]).to_csv(box_file)
dtd_stocks.sort_values(['name'],ascending=[True]).to_csv(data_file)

### Extra addition

In [44]:
sql = '''
SELECT name, YEAR(sells.date) AS sell_year, MONTH(sells.date) AS sell_month, sells.date AS sell_date,
qty, sells.price AS sell_price, buys.price AS buy_price,
sells.net AS sell_amt, buys.net AS buy_amt, profit,
ROUND((sells.price - buys.price)/buys.price*100,2) AS pct, sells.id AS sell_id, buys.kind
FROM sells JOIN buys ON sells.buy_id = buys.id
JOIN stocks ON buys.stock_id = stocks.id
WHERE YEAR(sells.date) = %s
AND name = '%s'
ORDER BY sells.date, name'''

sql = sql % (year, name)
print(sql)


SELECT name, YEAR(sells.date) AS sell_year, MONTH(sells.date) AS sell_month, sells.date AS sell_date,
qty, sells.price AS sell_price, buys.price AS buy_price,
sells.net AS sell_amt, buys.net AS buy_amt, profit,
ROUND((sells.price - buys.price)/buys.price*100,2) AS pct, sells.id AS sell_id, buys.kind
FROM sells JOIN buys ON sells.buy_id = buys.id
JOIN stocks ON buys.stock_id = stocks.id
WHERE YEAR(sells.date) = 2022
AND name = 'SIS'
ORDER BY sells.date, name


In [45]:
df_sells = pd.read_sql(sql, conpf)
df_sells.head()

,name,sell_year,sell_month,sell_date,qty,sell_price,buy_price,sell_amt,buy_amt,profit,pct,sell_id,kind
0,SIS,2022,1,2022-01-19,1500,42.75,40.50,"63,982.97","60,884.55","3,098.42",5.56,907,DTD
1,SIS,2022,3,2022-03-31,4500,40.75,40.50,"182,968.84","182,653.67",315.17,0.62,950,DTD
2,SIS,2022,5,2022-05-26,800,30.75,28.75,"24,545.52","23,050.94","1,494.58",6.96,968,DTD


In [46]:
ttl_by_month = df_sells.groupby(['sell_month'])['profit'].sum()
ttl_by_month

sell_month
1   3,098.42
3     315.17
5   1,494.58
Name: profit, dtype: float64

In [47]:
ttl_by_month = df_sells.groupby(['sell_year','sell_month','name'], as_index=True).agg(
    {
        'profit':['sum','count'],
    }
)
ttl_by_month

profit      
                               sum count
sell_year sell_month name               
2022      1          SIS  3,098.42     1
          3          SIS    315.17     1
          5          SIS  1,494.58     1

In [48]:
pd.set_option('max_rows',None)
ttl_by_month.get('profit')

sum  count
sell_year sell_month name                
2022      1          SIS  3,098.42      1
          3          SIS    315.17      1
          5          SIS  1,494.58      1

In [49]:
ttl_by_month.groupby(level='name').sum()

profit      
          sum count
name               
SIS  4,908.17     3

In [50]:
ttl_by_month.sum()

profit  sum     4,908.17
        count       3.00
dtype: float64

### Profit by Stock

In [51]:
#name = 'IMH'
sql = '''
SELECT name, sells.date AS sell_date,
qty, sells.price AS sell_price, buys.price AS buy_price,
sells.net AS sell_amt, buys.net AS buy_amt, profit,
ROUND((sells.price - buys.price)/buys.price*100,2) AS pct, sells.id AS sell_id, buys.kind
FROM sells JOIN buys ON sells.buy_id = buys.id
JOIN stocks ON buys.stock_id = stocks.id
WHERE YEAR(sells.date) = %s AND name = '%s'
ORDER BY sells.date DESC'''

sql = sql % (year, name)
print(sql)

df_name = pd.read_sql(sql, conpf)
df_name.style.format(format_dict)



SELECT name, sells.date AS sell_date,
qty, sells.price AS sell_price, buys.price AS buy_price,
sells.net AS sell_amt, buys.net AS buy_amt, profit,
ROUND((sells.price - buys.price)/buys.price*100,2) AS pct, sells.id AS sell_id, buys.kind
FROM sells JOIN buys ON sells.buy_id = buys.id
JOIN stocks ON buys.stock_id = stocks.id
WHERE YEAR(sells.date) = 2022 AND name = 'SIS'
ORDER BY sells.date DESC


,name,sell_date,qty,sell_price,buy_price,sell_amt,buy_amt,profit,pct,sell_id,kind
0,SIS,2022-05-26,800,30.75,28.75,"24,545.52","23,050.94","1,494.58",6.96%,968,DTD
1,SIS,2022-03-31,"4,500",40.75,40.50,"182,968.84","182,653.67",315.17,0.62%,950,DTD
2,SIS,2022-01-19,"1,500",42.75,40.50,"63,982.97","60,884.55","3,098.42",5.56%,907,DTD


In [52]:
profit = df_name.profit.sum()
cost = df_name.buy_amt.sum()
percent = round(profit/cost*100,2)
profit, percent

(4908.17, 1.84)

In [53]:
df_by_price = df_name.groupby(['sell_price'], as_index=True).agg(
    {
        'sell_price':['count'],
    }
)
df_by_price

,sell_price
,count
sell_price,
30.75,1
40.75,1
42.75,1


In [54]:
df_by_price = df_name.groupby(['sell_price','buy_price'], as_index=True).agg(
    {
        'sell_price':['count'],
    }
)
df_by_price

,,sell_price
,,count
sell_price,buy_price,
30.75,28.75,1
40.75,40.50,1
42.75,40.50,1
